In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import matplotlib.pyplot as plt
from tvdatafeed.tvDatafeed.main import *
import pandas_ta as pandas_ta
import Talib
from stockstats import StockDataFrame
import vectorbt as vbt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.subplots as ms
import warnings
import timeit
from Function import get_data
from Function import handleRaw

# Import from Tradingview (Old)

In [ ]:
# Tài khoản và mật khẩu
USERNAME = 'tradingpro.112233@gmail.com'
PASSWORD = 'Vuatrochoi123'

In [ ]:
def return_time(time):
    try:
       if time == "1h":
           return Interval.in_1_hour
       elif time =="15m":
           return Interval.in_15_minute
       elif time == "30m":
           return Interval.in_30_minute
       elif time == "1m":
           return Interval.in_1_minute
       elif time == "5m":
           return Interval.in_5_minute
       elif time == "4h":
           return Interval.in_4_hour
       elif time == "2h":
           return Interval.in_2_hour
       elif time == "3h":
           return Interval.in_3_hour
       elif time == "3m":
           return Interval.in_3_minute
       elif time == "45m":
           return Interval.in_45_minute
       elif time == "1d":
           return Interval.in_daily
       elif time == "1w":
           return Interval.in_weekly
       elif time == "1M":
           return Interval.in_monthly

    except:
        return "Unknown time format"

In [ ]:
tv = TvDatafeed(USERNAME, PASSWORD)

# Get historical data
input_message = 'Symbol: '
symbol = str(input(input_message))
input_message = 'Exchange: '
exchange = str(input(input_message))
input_message = "Interval_time ['1m', '3m', '5m', '15m', '30m', '45m', '1h', '2h', '3h', '4h', '1d', '1w', '1M']"
interval_time = str(input(input_message)) # Edit to user can change
n_bars = 20000 # No of bars to download, max 5000. Defaults to 

if not symbol:
  symbol = 'DCM'
  
if not exchange:
  exchange = 'HOSE'
if not interval_time:
  interval_time = '4h'
interval_time = return_time(interval_time)
### Download from multi sources
print('-------------------')
print(f"Symbol: {symbol} - Exchange: {exchange}")
df = tv.get_hist(symbol=symbol, exchange=exchange, interval=interval_time, n_bars=n_bars)

# Data VN Index:
historical_vnindex_price_df = tv.get_hist(symbol='VNINDEX', exchange='HOSE', interval=interval_time, n_bars=n_bars)
#historical_us_price_df = tv.get_hist(symbol='tsla', exchange='NASDAQ', interval=interval_time, n_bars=n_bars)
row_count, column_count = df.shape
print(f"Download done with {row_count} rows and {column_count} columns")
print(df.values.tolist())

ohlc = df

In [ ]:
df

# Get data from Metastock

In [ ]:
mongo_client = get_data.get_client()

In [ ]:
meta_data = get_data.get_datbase(mongo_client,data_source='metastock')
meta_data

In [ ]:
df = get_data.fetch_meta(symbol='VNINDEX',mongo_data=meta_data,
                           type_='EOD', kind='index',marketdivision='',
                           start='2000/01/01',stop='now')
df

In [ ]:
df = pd.read_csv(r'C:\\Users\\Admin\\OneDrive\\Note\\backtest.csv')

# Define some functions

In [ ]:
def pivotID(data, l, m, n):
    if l - m - n < 0 or l >= len(data):
        return 0
    
    pivotID_low = 1
    pivotID_high = 1
    
    for i in range(l - m - n, l + 1):
        if (data.low[l - m] > data.low[i]):
            pivotID_low = 0
        if (data.high[l - m] < data.high[i]):
            pivotID_high = 0
    if pivotID_low and pivotID_high:
        return 3
    elif pivotID_low:
        return 1
    elif pivotID_high:
        return 2
    else:
        return 0

In [ ]:
def RSIpivotID(data, l, m, n):
    if l - m - n < 0 or l >= len(data):
        return 0

    RSIpivotID_low = 1
    RSIpivotID_high = 1
    for i in range(l - m - n, l + 1):
        if(data.RSI[l - m] > data.RSI[i]):
            RSIpivotID_low = 0
        if(data.RSI[l -  m] < data.RSI[i]):
            RSIpivotID_high = 0
    if RSIpivotID_low and RSIpivotID_high:
        return 3
    elif RSIpivotID_low:
        return 1
    elif RSIpivotID_high:
        return 2
    else:
        return 0 

In [ ]:
def holding_period(entries):
    df2 = pd.DataFrame(entries).reset_index()
    df2 = df2.set_axis(['datetime','entry'], axis = 1, copy = False)
    df2['pos'] = pd.Series(dtype = float)
    df2['close'] = pd.DataFrame(df['close'].values)
    for a in df2.index:
        if df2.at[a, 'entry'] == True:
            df2.at[a, 'pos'] = 1
    df2['pos_shift'] = df2['pos'].shift(1)
    df2['check_entry'] = df2['datetime'][(~df2['pos'].isna()) & (df2['pos_shift'].isna())]
    df2['check_entry'] = df2['check_entry'].fillna(method = 'ffill')
    df2['holding'] = df2['datetime'] - df2['check_entry']
    df2['check_price'] = df2['close'][(~df2['pos'].isna()) & (df2['pos_shift'].isna())]
    df2['check_price'] = df2['check_price'].fillna(method = 'ffill')
    df2['return'] = (df2['close'] - df2['check_price'])/(df2['check_price'])
    df2= df2.drop(columns = ['check_price', 'pos', 'pos_shift', 'check_entry'])
    return df2

In [ ]:
stats = ['Start', 'End', 'Period', 'Start Value', 'End Value', 'Total Return [%]', 'Benchmark Return [%]',
         'Max Gross Exposure [%]', 'Total Fees Paid', 'Max Drawdown [%]', 'Max Drawdown Duration', 'Total Trades',
         'Total Closed Trades', 'Total Open Trades', 'Open Trade PnL', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
         'Avg Winning Trade [%]', 'Avg Losing Trade [%]', 'Avg Winning Trade Duration', 'Avg Losing Trade Duration',
         'Profit Factor', 'Expectancy', 'Sharpe Ratio', 'Calmar Ratio', 'Omega Ratio', 'Sortino Ratio']

# Get RSI Divergence and Loop with holding period for 1 stock / VNINDEX

In [ ]:
warnings.simplefilter('ignore', np.RankWarning)

In [ ]:
df1.index[0]

In [ ]:
def rsi_div(df):
    rsi = df.ta.rsi(close = df['close'], length = 14)
    df = pd.concat([df, rsi], axis = 1)
    
    df = df.rename(columns = {'RSI_14': 'RSI'})
    
    df['Pivot'] = df.apply(lambda x: pivotID(df, x.name, 5, 0), axis = 1)
    df['RSI_Pivot'] = df.apply(lambda x: RSIpivotID(df, x.name, 5, 0), axis = 1)
    
    backcdl = 80
    
    for j in range(df.index[0] + backcdl + 1, df.index[-1] + 1):
        maxim = np.array([])
        minim = np.array([])
        xxmin = np.array([])
        xxmax = np.array([])

        maximRSI = np.array([])
        minimRSI = np.array([])
        xxminRSI = np.array([])
        xxmaxRSI = np.array([])

        for i in range(j - backcdl, j + 1):
            if df.loc[i, 'Pivot'] == 1:
                minim = np.append(minim, df.iloc[i].low)
                xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
            if df.loc[i, 'Pivot'] == 2:
                maxim = np.append(maxim, df.iloc[i].high)
                xxmax = np.append(xxmax, i) # df.iloc[i].name
            if df.loc[i, 'RSI_Pivot'] == 1:
                minimRSI = np.append(minimRSI, df.iloc[i].RSI)
                xxminRSI = np.append(xxminRSI, df.iloc[i].name)
            if df.loc[i, 'RSI_Pivot'] == 2:
                maximRSI = np.append(maximRSI, df.iloc[i].RSI)
                xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

        slmin, intercmin = np.polyfit(xxmin, minim, 1)
        slmax, intercmax = np.polyfit(xxmax, maxim, 1)
        slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI, 1)
        slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI, 1)

        df.loc[j, 'Slope min'] = slmin
        df.loc[j, 'Slope max'] = slmax
        df.loc[j, 'Slope RSI min'] = slminRSI
        df.loc[j, 'Slope RSI max'] = slmaxRSI
    
    for i in df.index:
        if df.loc[i, 'Pivot'] == 1 or df.loc[i, 'RSI_Pivot'] == 1:
            if df.loc[i, 'Slope min'] < 0 and df.loc[i, 'Slope RSI min'] > 0:
                df.loc[i, 'Bull Signal'] = 100
            else:
                df.loc[i, 'Bull Signal'] = 0
        else:
            df.loc[i, 'Bull Signal'] = 0
        if df.loc[i, 'Pivot'] == 2 or df.loc[i, 'RSI_Pivot'] == 2:
            if df.loc[i, 'Slope max'] > 0 and df.loc[i, 'Slope RSI max'] < 0:
                df.loc[i, 'Bear Signal'] = -100
            else:
                df.loc[i, 'Bear Signal'] = 0
        else:
            df.loc[i, 'Bear Signal'] = 0
    
    return df

In [ ]:
df = rsi_div(df)
df

In [ ]:
df1 = df[500:1000]
df1

In [ ]:
df1 = rsi_div(df1)
df1

In [ ]:
df1[500:550]

In [ ]:
df = df.set_index('time')

In [ ]:
# Bullish Strategy with holding period
df_stats = pd.DataFrame(index = stats)
for m in (3, 5, 7, 10):
    entries = (df['Bull Signal'].shift(1) == 100)
    exits = (holding_period(entries)['holding'].dt.days >= m)
    
    bt_pf = vbt.Portfolio.from_signals(df['close'], entries, exits,  direction = 'longonly', accumulate = 'addonly',
                                           sl_stop = 0.06, sl_trail = True, tp_stop = 0.06, fees = 0.00, freq = '4h')
    a = bt_pf.stats()
    df_stats = pd.concat([df_stats, a], axis = 1)
df_stats

In [ ]:
# Bearish Strategy with holding period
df_stats = pd.DataFrame(index = stats)
for m in (3, 5, 7, 10):
    entries = (df['Bear Signal'].shift(1) == -100)
    exits = (holding_period(entries)['holding'].dt.days >= m)
    
    bt_pf = vbt.Portfolio.from_signals(df['close'], entries, exits,  direction = 'shortonly', accumulate = 'addonly',
                                           sl_stop = 0.06, sl_trail = True, tp_stop = 0.06, fees = 0.00, freq = '4h')
    a = bt_pf.stats()
    df_stats = pd.concat([df_stats, a], axis = 1)
df_stats

In [ ]:
# Plot candlestick chart and RSI line
df_pl = df[0:1000]
fig = ms.make_subplots(rows = 3, cols = 1, shared_xaxes = True, shared_yaxes = False, vertical_spacing = 0)
fig.append_trace(go.Candlestick(x = df_pl.index, open = df_pl['open'], high = df_pl['high'], low = df_pl['low'],
                                close = df_pl['close'], name = 'Candlestick'), row = 1, col = 1)
fig.append_trace(go.Scatter(x = df_pl.index, y = df_pl['RSI'], name = 'RSI'), row = 2, col = 1)
fig.update_layout(xaxis_rangeslider_visible = False)
fig.show()

In [ ]:
# Optimize SL, TP Strategy 1


entries = (df['Bull Signal'].shift(1) == 100)
exits = (holding_period(entries)['holding'].dt.days >= 3)
# entry_points = np.linspace(1,45, num=num)
# exit_points = np.linspace(55,99, num=num)

# grid = np.array(np.meshgrid(entry_points, exit_points)).T.reshape(-1,2)

# entries = rsi.rsi_crossed_below(10)
# exits = rsi.rsi_crossed_above(70)

sl_points = np.arange(0.1,1,0.1)
tp_points = np.arange(0.1,1,0.1)

grid = np.array(np.meshgrid(sl_points, tp_points)).T.reshape(-1,2)

sl_points = grid[:, [0]].flatten()
tp_points = grid[:, [1]].flatten()

sl = pd.DataFrame([sl_points],columns = [sl_points])
sl = sl.rename_axis(columns = 'stoploss')
tp = pd.DataFrame([tp_points],columns = [tp_points])
tp = tp.rename_axis(columns = 'takeprofit')

# grid = np.array(np.meshgrid(sl_points, tp_points)).T.reshape(-1,2)
# sl = rsi.rsi_crossed_below(list(grid[:, [0]]))
# tp = rsi.rsi_crossed_above(list(grid[:, [1]]))

step = 0.01  # 1%
stops = np.arange(step, 1 + step, step)

pf = vbt.Portfolio.from_signals(df['close'], entries, exits,  direction='longonly', accumulate='addonly', sl_stop = sl,
                                sl_trail = True, tp_stop = tp, fees = 0.00, freq = '4h')
metric = 'positions.win_rate'
pf_perf = pf.deep_getattr(metric)

pf_perf.vbt.heatmap(xaxis_title = 'StopLoss',yaxis_title = 'TakeProfit')